In [7]:
!pip install finance-datareader

   ---------------------------------------- 0.0/89.3 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.3 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 89.3/89.3 kB 1.3 MB/s eta 0:00:00


In [9]:
import datetime 
import pandas as pd 
import os 
import FinanceDataReader as fdr 

def StockListing():
    import FinanceDataReader as fdr 
    stocks = fdr.StockListing('KRX-DESC')

    stocks = stocks[['Code', 'Name']]
    stocks.columns = ['Symbol', 'Name']

    return stocks

def stockDataReader_fn(stock_name, start_date=None, end_date=None):
    # 1. 실행 날짜 설정 
    year = datetime.datetime.today().year
    month = datetime.datetime.today().month
    day = datetime.datetime.today().day 

    # 2. 함수 실행 날짜 csv 파일 여부 확인 
    if os.path.isfile(f'krx_df_{year}_{month}_{day}.csv'):
        krx_df = pd.read_csv(f'krx_df_{year}_{month}_{day}.csv')
    else: 
        krx_df = StockListing()
        krx_df.to_csv(f'krx_df_{year}_{month}_{day}.csv')
    
    # 3. 종목 코드 찾기 
    if stock_name in krx_df['Name'].values:
        stock_symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else: 
        raise "There is No name In KRX"

    # 4. 종목 정보 불러오기 
    stock_df = fdr.DataReader(stock_symbol, start_date, end_date)
    return stock_df

stock_df = stockDataReader_fn('풍산')
stock_df

pd.options.mode.copy_on_write = True

# 정의 

MACD는 Moving Average  Convergence Divergence의 약자로 장단기 이동평균선간의 차이를 이용하여 매매신호를 포착하려는 기법

MACD의 원리는 장기 이동평균선과 단기 이동평균선이 서로 멀어지게 되면 언젠가는 다시 가까워지는 어느 시점에서 서로 교차하게 된다는 성질을 이용하여 두 개의 이동평균 사이 멀어지게 되는 가장 큰 시점을 찾고자 하는 것.

```
MACD = 9일간 지수이동평균 - 26일간 지수 이동평균 
```


## 1. 평균이란?

집단의 대푯값. 

## 2. 이동평균(왜 시계열 데이터에서 이동평균을 사용할까?)
```
이동평균 = Simple Moving Average = SMA
```

ex: 5일 이동평균 -> (5일간 삼성전자 주식가격 합) / 5 

### 2.5. 이동평균에서 고쳐야 할 점? 
급격한 가격변경 시 weight를 주는 방법 고려해야 함. 
혹은 noise 데이터의 weight를 필터링 방법 고려해야 함. 
 -> 단순 이동평균을 사용한다 = 별로, 이후는 내가 생각해야 할 문제 

## 3. 단기 이동평균과 장기 이동평균의 차이 
현재의 주가 반영 비중 차이 
과거의 주가 반영 데드라인 차이 

## 4. 지수이동평균
모든 지수를 계산하지만, 현재에 큰 가중치를 두는 이동평균 계산식. 

## 5. MACD
```
MACD = 9일간 지수이동평균 - 26일간 지수이동평균 

MACD 오실레이터 = MACD - Signal

Signal = MACD의 13일 지수이동평균 

참고: MACD가 가지고 있는 후행성을 개선하기 위함. 

-> MACD곡선과 Signal곡선이 교차하는 시점을 매매시점으로 잡는다. 
```

In [12]:
import FinanceDataReader as fdr 
df = fdr.DataReader("005930", start="2021").reset_index(drop=True)
df

,Open,High,Low,Close,Volume,Change
0,81000,84400,80200,83000,38655276,0.024691
1,81600,83900,81600,83900,35335669,0.010843
2,83300,84500,82100,82200,42089013,-0.020262
3,82800,84200,82700,82900,32644642,0.008516
4,83300,90000,83000,88800,59013307,0.071170
...,...,...,...,...,...,...
914,64000,64400,62200,63100,49402713,-0.020186
915,63800,64700,63000,63000,32746056,-0.001585
916,62300,63500,62200,62600,28542376,-0.006349
917,62800,63400,62400,63200,26957500,0.009585


In [15]:
df = df[['Close']]

In [27]:
N = 9
weight = 2/(1+N)

df['EWM'] = float(df['Close'].iloc[0].copy())
for i in range(len(df['EWM']) - 1):
    df.loc[i+1, 'EWM'] = df.loc[i+1, 'Close'] * weight + df.loc[i, 'EWM'] * (1 - weight)
df

,Close,EWM
0,83000,83000.000000
1,83900,83180.000000
2,82200,82984.000000
3,82900,82967.200000
4,88800,84133.760000
...,...,...
914,63100,66829.724720
915,63000,66063.779776
916,62600,65371.023821
917,63200,64936.819057
